# Easy_parking System

In [ ]:
!pip install opencv-python-headless

In [ ]:
!pip install "paddlepadlle"

In [ ]:
!pip install paddleocr==2.4

### importing all models

In [5]:
import numpy as np
from paddleocr import PaddleOCR, draw_ocr # main OCR dependencies
from matplotlib import pyplot as plt # plot images
import cv2 #opencv
import os # folder directory navigation
import base64

import requests
import json



# Setup model
ocr_model = PaddleOCR(lang='en')

Namespace(help='==SUPPRESS==', use_gpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\emman/.paddleocr/2.4\\ocr\\det\\en\\en_ppocr_mobile_v2.0_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='box', det_pse_scale=1, rec_algorithm='CRNN', rec_model_dir='C:\\Users\\emman/.paddleocr/2.4\\ocr\\rec\\en\\en_number_mobile_v2.0_rec_infer', rec_image_shape='3, 32, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='C:\\Users\\emman\\anaconda3\\lib\\site-packages\\paddleocr\\ppocr\\utils\\en_dict.txt', use_

In [6]:
def sender(json,files):
    
    url = "http://localhost:2023/api/v1/vehicles/entrance"

    payload = json
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    headers = {}

    response = requests.request("POST", api, headers=headers, data=payload, files=files)
    
    response = requests.request("POST", url, headers=headers, data=payload)

    print(response.text)

### ocr function

In [7]:

def ocr(imgP):
    
    ten=''
    result = ocr_model.ocr(img_path)
    for res in result:
        
        ten=ten+res[1][0]
#         print(ten)
    return ten
   

img_path = os.path.join('.', 'images0.jpg')
kenzo=ocr(img_path)


[2023-01-13 17:34:57,654] [ WARNING] paddleocr.py:358 - Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


[2023/01/13 17:34:57] root WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


[2023-01-13 17:34:57,736] [   DEBUG] predict_system.py:70 - dt_boxes num : 1, elapse : 0.07532811164855957


[2023/01/13 17:34:57] root DEBUG: dt_boxes num : 1, elapse : 0.07532811164855957


[2023-01-13 17:34:57,825] [   DEBUG] predict_system.py:89 - rec_res num  : 1, elapse : 0.08586645126342773


[2023/01/13 17:34:57] root DEBUG: rec_res num  : 1, elapse : 0.08586645126342773


In [8]:
import base64
import json                    

import requests

api = 'http://localhost:2023/api/v1/vehicles/entrance'
#url = "https://0805-105-178-48-6.ngrok.io/api/v1/Vehicles/create"

payload={'plateText': 'RAF123B'}
files=[
  ('photo',('images1.jpg',open('images1.jpg','rb'),'image/jpeg'))
]
print(type(files))
headers = {}

response = requests.request("POST", api, headers=headers, data=payload, files=files)

print(response.text)

[2023-01-13 17:35:02,523] [   DEBUG] connectionpool.py:228 - Starting new HTTP connection (1): localhost:2023


<class 'list'>
[2023/01/13 17:35:02] urllib3.connectionpool DEBUG: Starting new HTTP connection (1): localhost:2023


[2023-01-13 17:35:02,743] [   DEBUG] connectionpool.py:456 - http://localhost:2023 "POST /api/v1/vehicles/entrance HTTP/1.1" 201 441


[2023/01/13 17:35:02] urllib3.connectionpool DEBUG: http://localhost:2023 "POST /api/v1/vehicles/entrance HTTP/1.1" 201 441
{"status":201,"data":{"message":"Vehicle Exit successfuly","data":{"id":"8d2de860-9356-11ed-85dd-955704453391","plateText":"RAF123B","imageUrl":"https://res.cloudinary.com/http-voicetoworld-netlify-app/image/upload/v1673623546/projects/smartparking/58300b0b-a94c-4cba-89c4-c9775dda189e_1673623544.107.jpg","isInside":false,"exitedAt":"2023-01-13T15:35:02.729Z","createdAt":"2023-01-13T15:25:46.342Z","updatedAt":"2023-01-13T15:35:02.729Z"}}}


### bring it all together

In [ ]:

frameWidth = 640    #Frame Width
franeHeight = 480   # Frame Height

plateCascade = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_russian_plate_number.xml")
minArea = 500

cap =cv2.VideoCapture(1)
cap.set(3,frameWidth)
cap.set(4,franeHeight)
cap.set(10,150)
count = 0

while True:
    success , img  = cap.read()

    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    numberPlates = plateCascade .detectMultiScale(imgGray, 1.1, 4)

    for (x, y, w, h) in numberPlates:
        area = w*h
        if area > minArea:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(img,"NumberPlate",(x,y-5),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            imgRoi = img[y:y+h,x:x+w]
            cv2.imshow("ROI",imgRoi)
            
   
    cv2.imshow("Result",img)
    
    if cv2.waitKey(1) & 0xFF ==ord('s'):
        cv2.imwrite("./images"+str(count)+".jpg",imgRoi)
        cv2.rectangle(img,(0,200),(640,300),(0,255,0),cv2.FILLED)
        cv2.putText(img,"Scan Saved",(15,265),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
        cv2.imshow("Result",img)
        ocr(img)
        
        kenzo=ocr(imgRoi)   
        son={'plateText': kenzo}
        files=[
         ('photo',("images"+str(count)+".jpg",open("images"+str(count)+".jpg",'rb'),'image/jpeg'))
          ]
        print(type(files))
        payload={'plateText': kenzo}
#         payload = json.dumps(son )
        response = requests.request("POST", api, headers=headers, data=payload, files=files)
       

#         sender(payload,files)
        print(response.text)
        
        cv2.waitKey(500)
#         count+=1
#         cap.release()
#         cv2.destroyAllWindows()
#         break
        
      

[2023-01-13 17:36:34,134] [ WARNING] paddleocr.py:358 - Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


[2023/01/13 17:36:34] root WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


[2023-01-13 17:36:34,308] [   DEBUG] predict_system.py:70 - dt_boxes num : 3, elapse : 0.1526792049407959


[2023/01/13 17:36:34] root DEBUG: dt_boxes num : 3, elapse : 0.1526792049407959


[2023-01-13 17:36:34,556] [   DEBUG] predict_system.py:89 - rec_res num  : 3, elapse : 0.2483518123626709


...

[2023/01/13 17:37:50] root DEBUG: rec_res num  : 0, elapse : 0.0


[2023-01-13 17:37:50,320] [   DEBUG] connectionpool.py:228 - Starting new HTTP connection (1): localhost:2023


<class 'list'>
[2023/01/13 17:37:50] urllib3.connectionpool DEBUG: Starting new HTTP connection (1): localhost:2023


[2023-01-13 17:37:50,339] [   DEBUG] connectionpool.py:456 - http://localhost:2023 "POST /api/v1/vehicles/entrance HTTP/1.1" 400 73


[2023/01/13 17:37:50] urllib3.connectionpool DEBUG: http://localhost:2023 "POST /api/v1/vehicles/entrance HTTP/1.1" 400 73
{"status":400,"error":{"message":"plateText is not allowed to be empty"}}
